# Chinook database
In this project, I am going to use SQL to answer business questions. I will be using the Chinook database, a sample database available for SQL Server, Oracle, MySQL. This database represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.


Below, you can observe a diagram of the database.

# IMAGE

In [11]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## List of tables in our database

In [12]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Number of tracks sold in the USA by genre

A record label has offered our Store four albums, and we can only include three albums out of the four in our catalogue. The albums are:
- Regal (Hip-Hop)
- Red tone (Punk)
- Meteor and the Girls (Pop)
- Slim Jim Bites (Blues)

This US record label is interested in the US market and has asked us if we could select the albums based on this criteria. Given this, we need to find which genres sell the most tracks in the USA.

In [47]:
%%sql
WITH 
usa_tracks_sold AS (
    SELECT *
      FROM invoice i
     INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
     WHERE c.country = "USA"
)

SELECT g.name Genre,
       COUNT(uts.track_id) AS Number_of_tracks,
       ROUND(CAST(100*COUNT(uts.track_id) AS FLOAT) / (SELECT COUNT(uts.track_id)
                              FROM usa_tracks_sold uts
                              ), 3) AS Percentage_sold
  FROM usa_tracks_sold uts
 INNER JOIN track t ON t.track_id = uts.track_id
 INNER JOIN genre g ON g.genre_id = t.genre_id

 GROUP BY g.name
 ORDER BY 2 DESC;





Done.


Genre,Number_of_tracks,Percentage_sold
Rock,561,53.378
Alternative & Punk,130,12.369
Metal,124,11.798
R&B/Soul,53,5.043
Blues,36,3.425
Alternative,35,3.33
Latin,22,2.093
Pop,22,2.093
Hip Hop/Rap,20,1.903
Jazz,14,1.332


As we can see on the table above, the least common genre in our store is Hip-Hop among the albums that we are considering. Based on this, we should include the other three albums.

##  Analysing employee productivity

We want to analyse the productivity of the Sales Support Agents within our company.

In [54]:
%%sql

SELECT e.first_name || " " || e.last_name name_employee,
       e.birthdate Birthday,
       hire_date Hire_date,
       SUM(i.total) Total_Sales
  FROM employee e
 INNER JOIN customer c ON c.support_rep_id = e.employee_id
 INNER JOIN invoice i ON i.customer_id = c.customer_id
 GROUP BY 1
 HAVING e.title = "Sales Support Agent";


Done.


name_employee,Birthday,Hire_date,Total_Sales
Jane Peacock,1973-08-29 00:00:00,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,1947-09-19 00:00:00,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,1965-03-03 00:00:00,2017-10-17 00:00:00,1393.920000000002


As we can see above, there is a 20% difference between Jane (top seller) and Steve (bottom seller). Nonetheless, this difference can be explained by the day the were hired.

## Analysisng Sales by Country

This time we would like to analyze the sales data for customers from each different country. Countries with only one client will be aggregated as "Other".

In [150]:
%%sql

WITH SalesbyCountry AS (
    SELECT 
           CASE
                WHEN COUNT(DISTINCT c.customer_id) = 1 THEN "Other"
                ELSE c.country
           END AS Country,
           CASE
                WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 0
                ELSE COUNT(DISTINCT c.customer_id)
           END AS sort,
           COUNT(DISTINCT c.customer_id) Customers,
           COUNT(i.invoice_id) Number_of_Sales,
           SUM(il.unit_price) Total
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    GROUP BY c.country
)

SELECT Country,
       SUM(Customers) Customers,
       ROUND(SUM(Total),3) Total_Sales,
       ROUND(SUM(Total)/SUM(Customers),3) Sales_per_Customer
  FROM SalesbyCountry
 GROUP BY 1
 ORDER BY sort DESC;





Done.


Country,Customers,Total_Sales,Sales_per_Customer
USA,13,1040.49,80.038
Canada,8,535.59,66.949
Brazil,5,427.68,85.536
France,5,389.07,77.814
Germany,4,334.62,83.655
United Kingdom,3,245.52,81.84
Czech Republic,2,273.24,136.62
India,2,183.15,91.575
Portugal,2,185.13,92.565
Other,15,1094.94,72.996
